# Scrape Youtube Comments

![](https://i.postimg.cc/x1Xv7xzY/ytb-scrape.jpghttps://i.postimg.cc/x1Xv7xzY/ytb-scrape.jpg)

#### This project will demonstrate how to scrape youtube comment for free without using Google API.

### GitHub Repo : [https://github.com/ahmedshahriar/youtube-comment-scraper](https://github.com/ahmedshahriar/youtube-comment-scraper) 

# Libraries & Configuration

In [1]:
"""
By default the below script will download most recent 100 comments of a youtube video
You can change the default filter (line 30 onwards)

Variables :
COMMENT_LIMIT : How many comments you want to download 
SORT_BY_POPULAR : filter comments by popularity (0 for True , 1 for false)
SORT_BY_RECENT : filter comments by recently posted (0 for True , 1 for false)
"""

import pandas as pd
import json
import os
import sys
import re
import time

import requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/watch?v=EY_Ddus5wic'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# csv file name
FILE_NAME = 'ytb_comments.csv'

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent False, change to 1 to download latest comments
SORT_BY_RECENT = 0
# set comment limit
COMMENT_LIMIT = 100

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

## Utility Function

In [3]:
def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)

def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return # Unable to extract configuration
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        # Comments disabled?
        return

    needs_sorting = sort_by != SORT_BY_POPULAR
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            yield {'cid': comment['commentId'],
                   'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                   'time': comment['publishedTimeText']['runs'][0]['text'],
                   'author': comment.get('authorText', {}).get('simpleText', ''),
                   'channel': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'photo': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'heart': next(search_dict(comment, 'isHearted'), False)}

        time.sleep(sleep)

def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)

In [4]:
def main(url):
    """
    This function will save the comments into a dataframe and output a csv file
    By default, it will append the comments in csv, not overwriting them, change it in line 34
    To preview the comments in json, uncomment the lines - 22 and 23
    """
    df_comment = pd.DataFrame()
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading Youtube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = df_comment.append(comment, ignore_index=True)

            # comments overview in json
            # comment_json = json.dumps(comment, ensure_ascii=False)
            # print(comment_json)

            count += 1

            if limit and count >= limit:
                break

        print("DataFrame Shape: ",df_comment.shape,"\nComment DataFrame: ")
        display(df_comment)

        if not os.path.isfile(FILE_NAME):
            df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
        else:  # else it exists so append without writing the header
            df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

# Save Comments

## Single Video Link

In [5]:
# dumping youtube comments

""" 
Dump comments to a csv  from a single video

"""
youtube_URL = 'https://youtu.be/A7yHrPoZ2mE'
main(youtube_URL)

DataFrame Shape:  (0, 0) 
Comment DataFrame: 


""



[2.46 seconds] Done!


In [6]:
df_comment.shape

NameError: name 'df_comment' is not defined

In [5]:
# df_comment = pd.read_csv('ytb_comments.csv')
# df_comment.shape

(20, 8)

## List of Links

In [6]:
"""
Dump to a csv from a a list with video links
"""
ytb_video_list = ['https://youtu.be/0sOvCWFmrtA',
                  'https://www.youtube.com/watch?v=TuIgtitqJho',
                  'https://www.youtube.com/watch?v=hinZO--TEk4',
                  'https://youtu.be/q6EoRBvdVPQ?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo']

for video_link in ytb_video_list:
    main(video_link)

DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgwxW_mKXXacgMFEkfl4AaABAg,I can't believe they've put this one youtube f...,9 maanden geleden,Human,UClAmpEDjU4dzJjOqZb_-iXA,"1,8K",https://yt3.ggpht.com/ytc/AMLnZu_YiKWfH1r6I1sX...,0.0
1,UgwkUAyXD5KXL5iGuPZ4AaABAg,Timestamps for entire course(this is copied fr...,7 maanden geleden (bewerkt),Maher Turifi,UCOS2GDYYqPp3DTrnDGUWMoQ,644,https://yt3.ggpht.com/ytc/AMLnZu9KfysHY__TBLAd...,0.0
2,Ugyq5RWPOnLokfPwcQd4AaABAg,People pay more than $2000 for Python lectures...,8 maanden geleden,Kaaluin,UCWaWYE-R6nKPQ9hyerQf0mA,206,https://yt3.ggpht.com/ytc/AMLnZu_EAH96GDMHdomn...,0.0
3,UgxWYu3mC-_p-kpxGZZ4AaABAg,The 7 dislikes are from those instructors sell...,9 maanden geleden,Mbm,UCUjY09GQF-sZZzGO4CgYVMQ,198,https://yt3.ggpht.com/ytc/AMLnZu8jmAdWc3tyxqGV...,0.0
4,UgyY8gIFypFESqvQFcp4AaABAg,"I was a carpenter before this video, now I am ...",2 maanden geleden,İrfan Baykara,UCanh_tGSXXa2u1zYVW6QF3w,61,https://yt3.ggpht.com/ytc/AMLnZu87BMbdTQFJHk0J...,0.0
5,UgzyvsB5n-y4JlahYcp4AaABAg,I'm going to watch this whole course 2 hours a...,9 maanden geleden,Nord Warrior,UCzDyyUmBb1GAMmX1eVndTpg,317,https://yt3.ggpht.com/ytc/AMLnZu-ei1h8uQNc2QIh...,0.0
6,UgzdlRWgcSPGGDZzVOh4AaABAg,"This is what I've been searching all day, and ...",9 maanden geleden,Syllight,UCr1PFONg4u14j-DfdggV4YA,197,https://yt3.ggpht.com/ytc/AMLnZu_9gt2WjgppEFfo...,0.0
7,UgwjDbssfQh3CSyK5B54AaABAg,As a developer he fulfilled his part for young...,9 dagen geleden (bewerkt),Watch and Enjoy104,UCHXPJMcI9UuBqL7bpCH__tQ,0,https://yt3.ggpht.com/ytc/AMLnZu8Dflevn8pJUpol...,0.0
8,Ugyy3d-FIOUll_Rsz6B4AaABAg,My team is currently transitioning from Django...,9 maanden geleden,Asuzu Kosi,UCj8gM_lLi0UtaSXQWW8muNg,108,https://yt3.ggpht.com/ytc/AMLnZu9mWQGDg8qXA6MK...,0.0
9,UgwqcTWOd2EWHfZhrgB4AaABAg,"I have 2 years experience in this, and genuine...",9 maanden geleden,Rahul Gupta,UCFRA_LOvBbDV66-WspB_YcA,40,https://yt3.ggpht.com/ytc/AMLnZu9zruTpYczbnbSK...,0.0



[1.34 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgwSDga4oAx1wOVzqxJ4AaABAg,If you want more stable coefficients when find...,1 jaar geleden,Abhishek Thakur,UCBPRJjIWfyNG4X-CRbnv78A,15,https://yt3.ggpht.com/kXhp0q_y7W_KDSjaTruJDdAX...,0.0
1,UgxOYfgYdcUbPF7UsUd4AaABAg,Thank you for the video. As usual amazing high...,1 jaar geleden,Abi K,UCu6hYINoiCOSwGPV24CurJA,0,https://yt3.ggpht.com/ytc/AMLnZu-xcuk38DVPTQUS...,1.0
2,UgwcJdH6EdWPws8KH5x4AaABAg,Awesome work !!! ❤️\nThankew so much sir 🙏\nE...,1 jaar geleden,shubham arora,UCnIpBGX5EgzL3txOku5dYGw,0,https://yt3.ggpht.com/ytc/AMLnZu-p8Zp7a_39b6I8...,0.0
3,Ugw1EHjhdiA2HWuRJ2N4AaABAg,Glad ...to see this sort of contents ...Believ...,1 jaar geleden (bewerkt),_AnanD_ ML_,UCd76XdJyTiiQVigypxm9Ksg,15,https://yt3.ggpht.com/N3RU69lPmccgjEPpsPlwG5UW...,0.0
4,UgzIpQBt3o988yVBy4V4AaABAg,Really cool video as always Abhishek!\nI actua...,1 jaar geleden (bewerkt),Manabendra Rout,UCFDdXd9Xgd07dUbRkM4Gp8A,0,https://yt3.ggpht.com/ytc/AMLnZu_wC6R5Z8nLWGj0...,0.0
5,UgxyN1ljiOE4wtmkREN4AaABAg,How do you do blending with time series split ...,1 jaar geleden (bewerkt),zubin mehta,UCdro_yGem44nA7hYwX0IIxw,0,https://yt3.ggpht.com/ytc/AMLnZu9FL607XtMKvfUI...,0.0
6,UgzetrQrGa0MGEqBpEJ4AaABAg,This is one of the most comprehensive tutorial...,2 maanden geleden,FloopyBits,UCNW1wZgng-nXu3yLgLNASzg,0,https://yt3.ggpht.com/ytc/AMLnZu9g61mAvxY2a2d7...,0.0
7,UgxQqhhVrI_5TTNYdIJ4AaABAg,Thank you for this sir!\nI can't wait to learn...,1 jaar geleden,Asogwa Kenneth,UCPYHtoZ2wOhB7sOFRAMquhQ,0,https://yt3.ggpht.com/ytc/AMLnZu_Hq74IFrOZRjrZ...,1.0
8,UgxgM8ZiGs_3WKH1a0R4AaABAg,"Hi Abhishek, thanks for the great content you ...",1 jaar geleden,Ambarish Kapil,UCFlNYOjiTQ3KHZBKE-YlYRQ,0,https://yt3.ggpht.com/ytc/AMLnZu_pGsMLCW6Lc7xU...,0.0
9,Ugwiy9y5UljANMYD8zF4AaABAg,Amazing video Abhishek. You keep emphasizing o...,1 jaar geleden,Vinay Tiwari,UCY0Cyn5xAbZKzPfEA978AUw,1,https://yt3.ggpht.com/ytc/AMLnZu_gTrTs7r9Z9mmq...,0.0



[1.10 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgyIDQFDtULyH1UJTeZ4AaABAg,Full code is available here: https://github.co...,2 jaar geleden,Abhishek Thakur,UCBPRJjIWfyNG4X-CRbnv78A,24,https://yt3.ggpht.com/kXhp0q_y7W_KDSjaTruJDdAX...,0.0
1,UgyI40uufIZdq8rTSAp4AaABAg,Great stuff as usual. Complexity simplified. T...,2 jaar geleden,shaheer zaman,UC_j9VcGDw01LjQoP-z9_jWw,3,https://yt3.ggpht.com/ytc/AMLnZu8ln4uasnYnLH9D...,1.0
2,UgwBCXQ03uvPReTxAzR4AaABAg,This is the most organized and neat implementa...,1 jaar geleden,Chiranshu Adik,UCi81GqwjWEfrN-jni7Sqgzg,11,https://yt3.ggpht.com/ytc/AMLnZu9fOOXxpjpkMTHv...,1.0
3,UgwK10Wgag_JSoqNCxt4AaABAg,"Great video as always, good to see you are cov...",2 jaar geleden,Mankaran Singh,UCYVS_RjnfJhou8SLZNvRaGA,5,https://yt3.ggpht.com/ytc/AMLnZu_Ft9Lhg1K9G2TB...,1.0
4,UgzK3EULuWCPweQyNmF4AaABAg,"Hi, thank you for a very helpful video! I have...",1 jaar geleden,Michelle Belgrave,UCb59PEXQcInh6jGk9ublW3g,0,https://yt3.ggpht.com/ytc/AMLnZu89YjqQ4he4QG6j...,0.0
5,UgynJEWRWwF2eHqrd-x4AaABAg,Thank you for your extremely helpful and infor...,2 jaar geleden,HIEU NGO TRUNG,UCIzydXGh-HtiDg8AEf3HsXw,0,https://yt3.ggpht.com/ytc/AMLnZu-kMCn5SqswxRhB...,1.0
6,UgwCBa8i9EcDTyaPrCN4AaABAg,"This is SUCH a nice video. Thanks, Abhishek.\n...",2 jaar geleden (bewerkt),Raama V.,UC7ZBIpcm8_WSeHQT96RT5NQ,0,https://yt3.ggpht.com/ytc/AMLnZu_sGsP7vJ1AubNV...,0.0
7,UgxDaEe3cggNR-TrdF14AaABAg,amazing stuff! truly end-to-end with no extern...,1 jaar geleden,whatever...,UCyuo9sKMV-_8OoP_aftcf9A,3,https://yt3.ggpht.com/WdE8sruYHwg9IMqZfeckyRgS...,1.0
8,UgxOWE48PqjCrDwyiB54AaABAg,"Hey Abhiskek, grateful for this content. A que...",2 jaar geleden (bewerkt),Akshay Raj,UC6Fq5nlGrP4UjUoa92Pf6MQ,0,https://yt3.ggpht.com/ytc/AMLnZu-ZQMKgZ_aGvRtS...,0.0
9,UgxiwK0N7VJd469Mgol4AaABAg,Thanks for the great video. You mentioned abou...,2 jaar geleden,Ayuumi,UCNIzB4sSDz0VebM0qusbyFA,0,https://yt3.ggpht.com/ytc/AMLnZu-ZAIJ5sZEfm3JH...,1.0



[0.97 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgwrrqPVIL-MN2MSHsp4AaABAg,Theres no evidence that dinosaurs didn’t actua...,2 jaar geleden,R SS,UCxVKkYRnsOSdYKEaNUwFG-g,43K,https://yt3.ggpht.com/ytc/AMLnZu-zu7sE3oGqw_N1...,0.0
1,UgzjYqysZKRT3yMN-sN4AaABAg,"To date, the only meme I never fully understoo...",1 maand geleden,PuzzLEGO,UC4GaWcWGhh5n1WmJvb1oBkA,"6,4K",https://yt3.ggpht.com/qMc36j22XaxCPe58-4jzWA86...,0.0
2,Ugy44QaOxgV4w8UmhNF4AaABAg,My favorite part about this video is how when ...,6 maanden geleden,Dylan,UCyPZ2aq7WrLdOrRtvI3FiDw,11K,https://yt3.ggpht.com/ytc/AMLnZu_ajYpnzAUTJUWc...,0.0
3,UgyExurNBjW1n2iJddh4AaABAg,Ya know your life is sad when you have nostalg...,3 weken geleden,05car Magu1re,UCNcrXccs8SBKLwxVuLZ0iIA,115,https://yt3.ggpht.com/aNwHxFTNW8QOrDQScNMI1U3F...,0.0
4,Ugyntn0vft-ybnC5CKR4AaABAg,"Fun fact, This is an elegant meme, from a more...",4 maanden geleden,General_Cheese6,UCqau0K0yGXmpps7-MsJLluw,"2,1K",https://yt3.ggpht.com/9Ohed-a3BMZUwwmSZDZVngS3...,0.0
5,UgyuUFLuSHhlutgd4Wp4AaABAg,I loved the part when the T-rex-looking dinosa...,5 dagen geleden,Paotato,UC7HS0wDoN0duvLbJjo9lWCA,109,https://yt3.ggpht.com/kqZ4eQ9Z-N_LpJXEUv6G7Mqj...,0.0
6,UgwwKTurLn0_IBK5jl14AaABAg,I can't believe YouTube legitimately recommend...,1 jaar geleden,Robo Puff,UC08V2vqxi_vVb48LHOdiM7g,"5,8K",https://yt3.ggpht.com/QkHdcMpaI62Lbv0d-4ytFju0...,0.0
7,Ugxe3ZlH7h6_uyAB8NJ4AaABAg,This meme has existed for 10 years…\n\n\nIt’s ...,1 maand geleden,MCEnthusiast,UCayRAbdbdVI8S0ehhbN9NFA,"3,2K",https://yt3.ggpht.com/zam6xFd6gV9HZEPxKlcKoAeZ...,0.0
8,UgxnmExmA0UtgaQR9Lh4AaABAg,I want this played at my funeral. So everyone ...,1 maand geleden,Rule 34,UC5zDKVXPjZq-kbpC2qtkOcw,"1,1K",https://yt3.ggpht.com/ytc/AMLnZu-wF2eVDnt_4PdI...,0.0
9,UgytDD4IDFAhqawQSch4AaABAg,"Even after ten whole years, this remains one o...",9 dagen geleden,Kalemaree,UC5l0jXZmVj3c9BQEmeuMuIA,34,https://yt3.ggpht.com/ytc/AMLnZu-dQCBZIOiy50aC...,0.0



[1.12 seconds] Done!


In [7]:
df_comment = pd.read_csv('./ytb_comments.csv')
df_comment.shape

(100, 8)

## Links From DataFrame

In [8]:
"""
Dump comments to a csv by parsing links from a csv with video links

Example -
Create a csv with one column titled 'link'
a sample is given below

'ytb_video_list.csv'

link
https://www.youtube.com/watch?v=-t_uhBBDbA4
https://www.youtube.com/watch?v=75vjjRza7IU
https://www.youtube.com/watch?v=j6dmaPzOBHY
https://www.youtube.com/watch?v=Yj2efyQV1RI
https://www.youtube.com/watch?v=HV652F7U6Qs
https://www.youtube.com/watch?v=47iXEucg3eo
https://www.youtube.com/watch?v=ofHXBLEE3TQ
https://www.youtube.com/watch?v=X6lGqSfVRT8
https://www.youtube.com/watch?v=a_-z9FhGBrE
https://www.youtube.com/watch?v=wTUM_4cVlE4


"""

youtube_data = pd.read_csv("../input/youtube-scrapped-data/YouTube_data.csv")
youtube_data.url[:3].map(lambda x: main(x))

DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgxKzzUbMWqiOMhUSRt4AaABAg,I am pretty sure ..... GATE has never seen top...,2 jaar geleden,Coding Intelligence,UC9be75EpkzViTlf4IYrDF1w,213,https://yt3.ggpht.com/ytc/AMLnZu81Hay_0pco2j_m...,1.0
1,Ugx9y759xm91IUFYBpZ4AaABAg,His simplicity is as great as marks.He has act...,2 jaar geleden,shiwank agrahari,UCHlhhjYwI3A_wccHNlIrFeg,79,https://yt3.ggpht.com/ytc/AMLnZu9cu9R9L0Qr4TdV...,1.0
2,UgwYSBp2hAQ9wEFm3h14AaABAg,"Book recommendations 02:44, 04:38\nFinished hi...",2 jaar geleden (bewerkt),GOT7's Yeojachingu,UCJWNTmGzELxGoqItiSZ34jw,24,https://yt3.ggpht.com/ytc/AMLnZu9g4o4A-orJiPUt...,0.0
3,UgxlkbOux8iI9gvVBtN4AaABAg,Thanks to made easy for interviewing toppers. ...,2 jaar geleden,Yogesh Kumar,UChTmHQ6DKgIFNlz62Q1AU7g,50,https://yt3.ggpht.com/ytc/AMLnZu9MeL7W3WzYSCPI...,1.0
4,UgwEFwjXrf74DgBfiNF4AaABAg,"You really worked hard and it payed off, still...",2 jaar geleden,CRACK GATE CSE,UCBqhJVfeXpfjXmouExfdzpQ,17,https://yt3.ggpht.com/ytc/AMLnZu8lDYPBlgdJdnw_...,0.0
5,Ugxnx7GBMnQccTJ370p4AaABAg,The questions were right on mark what aspiring...,2 jaar geleden,Aman Chaudhary,UCDvQ4evF5yHY6pGK3H9eU9g,8,https://yt3.ggpht.com/ytc/AMLnZu9lOLvTz9wXK1Dl...,1.0
6,UgzxBH8fxypIRPKpm614AaABAg,Very much inspiring and thank you so much for ...,1 jaar geleden,Priyanka,UCM75MK2r8WabtkFVfGISvyw,6,https://yt3.ggpht.com/ytc/AMLnZu8BnMzaSIh9cO4m...,0.0
7,UgxuFdYAw41IAJPDXQB4AaABAg,I know this guy. He was my friend in USICT inf...,1 jaar geleden,NAMAN GUPTA,UCEFHISIAMSrzCkdM7bFndYg,6,https://yt3.ggpht.com/ytc/AMLnZu_k-Xgp7TkdUiuY...,0.0
8,UgyUPIDPmop5O_SEF_h4AaABAg,He is exceptional and motivation also,2 jaar geleden,Rituraj Patel,UC-n-dL2Eh6ac6tHDd41gCYA,11,https://yt3.ggpht.com/ytc/AMLnZu_4bSpHkfibf9sr...,1.0
9,UgzTBA8juamXMFNyvGJ4AaABAg,Congrats bro...u made USICT proud 🤘,2 jaar geleden,Shubham Yadav,UC3bDf1BaJhOxCP7azNWUccw,11,https://yt3.ggpht.com/ytc/AMLnZu9mRcUaQMuz7ruY...,1.0



[1.01 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgxH00BrMnWXWG3vtB14AaABAg,"Stream ""WEEKS"" 💪: http://kevingates.lnk.to/weeks",1 jaar geleden,kevingatesTV,UCj2GTFekdV3EUsTVN8oaEqA,276,https://yt3.ggpht.com/ytc/AMLnZu-u76SNdXoRzYzc...,0.0
1,UgzRIW655xqGA86_O9p4AaABAg,I pray whoever reads this becomes successful,2 jaar geleden,Da Trillest,UC5ODfIBG8SV1smeBKkex1mA,"2,8K",https://yt3.ggpht.com/ytc/AMLnZu-l8E9kBClr9BFR...,0.0
2,UgwDycEbYx8CP8sWBTh4AaABAg,You will never know how much you helped me cha...,6 maanden geleden,No Excuses,UCWGq2MqSFKBldY1Wfy_rq3g,4,https://yt3.ggpht.com/K73UDcqawHe1HNFVMvUjsrXT...,0.0
3,UgwugIuhyH_VvXOq7P54AaABAg,"Relationships on the wrong side, left old frie...",2 jaar geleden,80sGyrl Xoxo,UClwGB3uKCkysdCHezrVimgw,419,https://yt3.ggpht.com/ytc/AMLnZu-l9n1_b6913Gz1...,0.0
4,UgxGp-16NOtXodJPLXB4AaABAg,He doesn’t get praised enough for being a lyri...,1 jaar geleden,Joe Schmo,UC6vsW1wFzjv7WpSgTpZ1mfQ,300,https://yt3.ggpht.com/ytc/AMLnZu-Rvqf7xaHaRlif...,0.0
5,Ugz4dry_g-otmb6La6J4AaABAg,"“I Had To Plowed Through The Roughage, I Cried...",10 maanden geleden,Caleb Fisher,UC9uuSx2wUN05mhLdzS3PaCw,67,https://yt3.ggpht.com/ytc/AMLnZu991MDJfBKQzkAP...,0.0
6,UgyvV3cAC87l0mRtY214AaABAg,This song just hits different. I play this dai...,2 jaar geleden,Clam Slam™©,UCU1AKox-o7iK_B9wMtrAFAw,34,https://yt3.ggpht.com/79pNJCtKUKkjXfvT5OiF2wqb...,0.0
7,Ugxq4LbOhBJwR4i9avJ4AaABAg,"R.I.P Grandma, I feel this song to the core. L...",1 jaar geleden,Kevin Skodowski Jr.,UCcv6BWJ9vVpGha_H3osdZ1w,18,https://yt3.ggpht.com/ytc/AMLnZu_8t-doqfsl16IV...,0.0
8,UgzQ4vMo8V0oXrbS6i54AaABAg,"Man, cant count the times when i didnt feel go...",2 jaar geleden,Mohamed Kurdieh,UCjGXXBNzzoN_O4xl_yMmWrg,58,https://yt3.ggpht.com/ytc/AMLnZu8eot3r5oj5msYC...,0.0
9,UgxKHE53slczVe4pSQ14AaABAg,Deserves a Grammy let's get them hits up there...,2 jaar geleden,Nikki Knight,UClwm_qpbZ5SxLqk-0ejUMyQ,19,https://yt3.ggpht.com/ytc/AMLnZu9h3ZNvKnqxR5sX...,0.0



[1.47 seconds] Done!
DataFrame Shape:  (13, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,Ugy8fR7USwVybD57f9x4AaABAg,Sir very Nice explaination,2 jaar geleden,V BANSAL,UCIlRGFAelfRO5YuwtkjZjBw,1,https://yt3.ggpht.com/ytc/AMLnZu_dE0Nvu2UqGwkV...,0.0
1,UgzMABScFUnsF5UmuT94AaABAg,Nice sir,2 jaar geleden,Vikash Meel,UCRi1r33q8J3v558zLr4GR4w,1,https://yt3.ggpht.com/ytc/AMLnZu_lsYo4PK2y1OOr...,0.0
2,Ugzqxivcb5-DZMdD2gV4AaABAg,Difficulty level,2 jaar geleden,Sunil Yadav,UC_oIRWah_Hy_CtqWVgD4vkg,1,https://yt3.ggpht.com/ytc/AMLnZu_In5iUAETlUeIV...,0.0
3,Ugx8BSpK9OWrawukSv54AaABAg,Sir iska ans diamond with double/bold border?,2 jaar geleden,Puskin Mittal,UC6zVmBLJlYABc871-Jsg70A,0,https://yt3.ggpht.com/ytc/AMLnZu-sGm1jrzJGTx5J...,0.0
4,UgyOQsgxGFCfkskffMF4AaABAg,Thank you sir,2 jaar geleden,Varsha Singh,UC27t8_kNyH7GdIMc3b6Nx3A,0,https://yt3.ggpht.com/ytc/AMLnZu_nCf_vRtiZ_qLK...,0.0
5,UgzaB_BJW1MC0SnkXAd4AaABAg,Very good. Detailed analysis,2 jaar geleden,Atomic S.H.,UC3lnQysASQ0bJqR1bVbpCtA,2,https://yt3.ggpht.com/ytc/AMLnZu-1pKkvBFAh8CfU...,0.0
6,Ugzemjyj50YE3SICxP94AaABAg,Sir profit % ke liye upon me 3000 kyo Ni liya,2 jaar geleden,sum 123,UC0Ot_ykaPD36fIs-UjkHoIw,0,https://yt3.ggpht.com/ytc/AMLnZu-B9zyprUJNrUw_...,0.0
7,UgzHtLMgFqRe2m05yG94AaABAg,Need to calculate tat,2 jaar geleden,Kodali prasanna,UCx7QfOiik1fCTzMrrMBuuJQ,1,https://yt3.ggpht.com/ytc/AMLnZu8weqIzwzMVSnQc...,0.0
8,UgxprQedWRRvTILypJd4AaABAg,Nice solution sir rahul Yadav gec bharatpur,2 jaar geleden,Rahul kumar yadav,UCxzRcy7GbqaqD6ldfXJ5VeA,2,https://yt3.ggpht.com/ytc/AMLnZu9VdEI4HC1mvZb1...,0.0
9,Ugz2p-2qRCzycEEvUmB4AaABAg,good afternoon sir .,2 jaar geleden,Mukesh Kumar,UCKZsrtIZX08tCzyj7VciGyg,2,https://yt3.ggpht.com/ytc/AMLnZu9iZ64VOeXq0cjb...,0.0



[0.94 seconds] Done!


0    None
1    None
2    None
Name: url, dtype: object

## Preview Output CSV

In [9]:
df_comment = pd.read_csv('./ytb_comments.csv')

print(f"{df_comment.shape[0]} rows, {df_comment.shape[1]} columns")
df_comment

153 rows, 8 columns


,cid,text,time,author,channel,votes,photo,heart
0,UgwtAC15ZpDsqRFscuB4AaABAg,I have a feeling we aren't too far from Disney...,7 maanden geleden,Stefan,UCn_cd_kyCntJs4yu249FayQ,171,https://yt3.ggpht.com/ytc/AMLnZu9dxMjVLZVsbgtJ...,1.0
1,Ugxl-2CmqdAsscaV9FJ4AaABAg,That’s incredible. I can’t believe how much wo...,7 maanden geleden,Tim Moeller,UCLjV7s8SwS3STCvWCZ_Znqw,143,https://yt3.ggpht.com/ytc/AMLnZu9wbE8S25hK6mxQ...,1.0
2,UgyUlKkXd9B07ojet-x4AaABAg,So awesome. Imagine when companies start doing...,7 maanden geleden,Neoux,UCXpCZq97ccTBtxWmYjRUFqQ,35,https://yt3.ggpht.com/gshUHXZNE6nn3Bu5ayvNg9MC...,1.0
3,Ugxjl5CzAJ928JZ6pdt4AaABAg,I love stuff like this because I feel like art...,11 maanden geleden,Jack Middleton,UCHNIRmyVDh52gm60BDEzyCw,87,https://yt3.ggpht.com/ytc/AMLnZu92mQwpJfkOX2Qh...,1.0
4,UgxbjRJpN8I_Va-LZiV4AaABAg,That's amazing! Just imaging what they'll be a...,7 maanden geleden,Kevin Buchanan,UCAvzjfD3s4urs4SVCvttjqQ,59,https://yt3.ggpht.com/ytc/AMLnZu8qfBRytlxiJCS1...,1.0
5,UgxSGTgzQNP0B0e7OPh4AaABAg,That…was…EPIC! Seeing that live would have bee...,7 maanden geleden,Isaac Sheppard,UC-Kkc303MEP_1heObyPye8Q,49,https://yt3.ggpht.com/ytc/AMLnZu8nVB2_P5WkAx-_...,1.0
6,Ugx2C0yrSoQPKUlav114AaABAg,"Deserves more attention. Amazing footage, Than...",7 maanden geleden,Freqenc,UCiGkSiKiWWoH2tUkLMEen9A,35,https://yt3.ggpht.com/ytc/AMLnZu_tH9f6RnanCXBf...,1.0
7,Ugxf-3kOeZE7mIe-kuJ4AaABAg,I shoot fireworks professionally and have a fe...,7 maanden geleden,1gallimaufry,UCOlbNpePQz9fp_Odwnpox6A,40,https://yt3.ggpht.com/ytc/AMLnZu-XEfAuPaQbDbcx...,1.0
8,UgwRjG43F7PJeoHEwLl4AaABAg,John Williams's contribution to cinema is one ...,7 maanden geleden,R Menon,UCO43lDqwCH_gWw2NZRw0K4w,31,https://yt3.ggpht.com/ytc/AMLnZu_ayCh0psrvAafK...,1.0
9,Ugz3-orRaUo173l7EPV4AaABAg,That was amazing to see the drones programmed ...,7 maanden geleden,dmonsef,UCf08iO4SyL-mbcE4Nvj8vlg,7,https://yt3.ggpht.com/ytc/AMLnZu9eZ8k9Hbt0RHX2...,1.0


## Reference

1. [https://github.com/egbertbouman/youtube-comment-downloader](https://github.com/egbertbouman/youtube-comment-downloader)